# Deploy a Model Version from a Different Account

Use this notebook on the dev/stage/prodcution accounts to deploy a Model Version from the shared `Model Registry` AWS Account.

In [ ]:
import boto3
import time
import datetime
import pandas as pd
from sagemaker import get_execution_role
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

## Prepare data for model inference

We load the California housing dataset from sklearn, and will use it to invoke SageMaker Endpoint

In [ ]:
data = fetch_california_housing()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

# we don't train a model, so we will need only the testing data
testX = pd.DataFrame(X_test, columns=data.feature_names)

In [ ]:
client = boto3.client("sagemaker")

In [ ]:
model_package_group_arn = "<MODEL PACKAGE GROUP ARN FROM THE SHARED AWS ACCOUNT>"

## View Model Groups and Versions

In [ ]:
list_model_packages_response = client.list_model_packages(ModelPackageGroupName=model_package_group_arn)
list_model_packages_response

## View Latest model version

In [ ]:
model_version_arn = list_model_packages_response['ModelPackageSummaryList'][0]['ModelPackageArn']
print(model_version_arn)

## View latest Model Version Details

In [ ]:
client.describe_model_package(ModelPackageName=model_version_arn)

## Deploy the Model from the Registry

In [ ]:
model_name = 'DEMO-sklearn-califonia-housing-model-' + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print("Model name : {}".format(model_name))

In [ ]:
primary_container = {
    'ModelPackageName': model_version_arn,
}

## Create `Model` 

In [ ]:
create_model_respose = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = get_execution_role(),
    PrimaryContainer = primary_container
)

print("Model arn : {}".format(create_model_respose["ModelArn"]))

## Create an Endpoint Config from the latest model

In [ ]:
endpoint_config_name = "sklearn-endpoint-config-" + datetime.datetime.now().strftime(
    "%Y-%m-%d-%H-%M-%S"
)

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTrafficVariant",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.c5.large",
            "InitialVariantWeight": 1,
        },
    ],
)

endpoint_config_response

## Deploy the Endpoint Config to a real-time endpoint

In [ ]:
endpoint_name = "sklearn-endpoint-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

create_endpoint_response

## Wait for Endpoint to be ready

In [ ]:
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

## Invoke Endpoint with `boto3`

In [ ]:
runtime = boto3.client("sagemaker-runtime")

In [ ]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

print(response["Body"].read())

## Clean up

Endpoints should be deleted when no longer in use, since (per the [SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/)) they're billed by time deployed.

In [ ]:
client.delete_endpoint(EndpointName=endpoint_name)